# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config_vacationpy import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
weather_info = pd.read_csv("../WeatherPy/weather_df.csv")
weather_info

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,chokurdakh,70.63,147.92,6.91,95,100,6.42,RU,1604083875
1,atuona,-9.80,-139.03,77.41,73,100,12.73,PF,1604083876
2,punta arenas,-53.15,-70.92,55.40,62,75,33.33,CL,1604083818
3,iqaluit,63.75,-68.51,3.20,65,20,5.82,CA,1604083974
4,bachaquero,9.97,-71.15,87.98,60,76,5.46,VE,1604083994
...,...,...,...,...,...,...,...,...,...
540,salinopolis,-0.61,-47.36,84.38,66,63,11.92,BR,1604083930
541,huarmey,-10.07,-78.15,68.04,70,100,8.99,PE,1604084573
542,anloga,5.79,0.90,82.40,78,20,8.05,GH,1604084106
543,claveria,18.61,121.08,78.42,83,60,19.69,PH,1604084126


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
#Congiure gmaps
gmaps.configure(api_key=g_key)

#Store lat/lng in location
location = weather_info[["Lat", "Lng"]]

#Add humidity
humidity = weather_info["Humidity"]

In [22]:
#Add points to map
fig = gmaps.figure(center=(45.0, -5.0), zoom_level=2)
max_humidity = np.max(humidity)
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
#Add heat layer
heat = gmaps.heatmap_layer(
    location, 
    weights = humidity, 
    dissipating = False,  
    point_radius=2
    )
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
